In [1]:
import string
UPPER_ALPHABET = string.ascii_uppercase
LOWER_ALPHABET = string.ascii_lowercase

### ROT13
ROT13 («rotar 13 posiciones») es un sencillo cifrado César utilizado para ocultar un texto sustituyendo cada letra por la letra que está trece posiciones por delante en el alfabeto. A se convierte en N, B se convierte en O y así hasta la M, que se convierte en Z. Luego la secuencia se invierte: N se convierte en A, O se convierte en B y así hasta la Z, que se convierte en M.

![Alt text](rot13.png)

In [2]:
def rot13(s: str):
    result = ""
    for letter in s:
        if letter.lower() in LOWER_ALPHABET:
            pos = (abs(ord(LOWER_ALPHABET[0]) - ord(letter.lower())) + 13) % 26
            result += LOWER_ALPHABET[pos].upper() if letter.isupper() else LOWER_ALPHABET[pos]
        else:
            result += letter
    return result

rot13("cvpbPGS{arkg_gvzr_V'yy_gel_2_ebhaqf_bs_ebg13_hyLicInt}")

"picoCTF{next_time_I'll_try_2_rounds_of_rot13_ulYvpVag}"

### Caesar

El cifrado César es una de las técnicas de cifrado más simples y más usadas. Es un tipo de cifrado por sustitución en el que una letra en el texto original es reemplazada por otra letra que se encuentra un número fijo de posiciones más adelante en el alfabeto. Por ejemplo, con un desplazamiento de 3, la A sería sustituida por la D (situada 3 lugares a la derecha de la A), la B sería reemplazada por la E, etc.

-> **Decrypt**: picoCTF{dspttjohuifsvcjdpoabrkttds}

In [3]:
def decrypt_caesar(s: str, i: int = 1):
    """decrypt a string using the Caesar cipher"""
    if str(input("You want to stop? [y/n]: ")) == 'y':
        return
    else:
        result = ""
        for letter in s:
            pos = (abs(ord(LOWER_ALPHABET[0]) - ord(letter)) - i) % 26
            result += LOWER_ALPHABET[pos]
        print(f"Message Decrypted: [{result}], with {i} shift")
        decrypt_caesar(s, i+1)

decrypt_caesar("dspttjohuifsvcjdpoabrkttds")

Message Decrypted: [crossingtherubiconzaqjsscr], with 1 shift
Message Decrypted: [bqnrrhmfsgdqtahbnmyzpirrbq], with 2 shift
Message Decrypted: [apmqqglerfcpszgamlxyohqqap], with 3 shift


### Easy1

A diferencia del cifrado César, que es un cifrado de sustitución monoalfabética, el cifrado Vigenère utiliza múltiples alfabetos (polialfabético) para cifrar el texto, lo que lo hace más resistente a los ataques de frecuencia. El cifrado Vigenère utiliza una palabra o frase como clave. Se asigna un número a cada letra de la clave y al mensaje original, y para ello se puede usar una tabla de correspondencia, donde A=0, B=1, C=2, ..., Z=25. Para descifrar el mensaje, el receptor debe conocer la clave y realizar una resta modular para obtener el mensaje original. Dado que tenemos una palabra como clave y una tabla, podemos deducir que este caso corresponde al cifrado Vigènere.

> Decrypt: `UFJKXQZQUNB` using this key: `SOLVECRYPTO` 

In [4]:
KEY, S = "SOLVECRYPTO", "UFJKXQZQUNB"
alphabet_dict = {chr(ord(UPPER_ALPHABET[0]) + i): i for i in range(26)}

def decrypt_vigenere(key: str, s: str):
    """Decrypt a string using Vigenere cypher"""
    result = ""
    for i, letter in enumerate(s):
        x = alphabet_dict.get(letter) - alphabet_dict.get(key[i])
        if x >= 0:
            result += UPPER_ALPHABET[x % 26]
        else:
            result += UPPER_ALPHABET[(x + 26) % 26]
    return result

print(decrypt_vigenere(KEY, S))

CRYPTOISFUN


### Spelling-quiz

Este parece un caso de cifrado César, pero que en lugar de descifrar una secuencia de letras, descifraremos una secuencia de números. Para descifrar un cifrado césar debemos hacer la operación inversa a $(x+k)\%26$, es decir $(x-k)\%26$; donde $k$ es la llave que indica el desplazamiento, y por lo tanto lo que queremos hallar. Para encontrar $k$ haremos un ataque de fuerza bruta hasta encontrar el valor de desplazamiento correcto.

![!random numbers](the_numbers.png)

In [5]:
sequence = [16, 9, 3, 15, 3, 20, 6, 20, 8, 5, 14, 21, 13, 2, 5, 18, 19, 13, 1, 19, 15, 14]
number_dict = {i: chr(ord(LOWER_ALPHABET[0]) + i) for i in range(26)}

def decrypt_numbers(seq, i=0):
    if str(input("You want to stop? [y/n]: ")) == 'y':
        return
    else:
        result = ""
        for number in seq:
            result += number_dict[(number-i) % 26].upper()
        print(f"Decrypted result: {result} with {i} shift")
        decrypt_numbers(seq, i+1)

decrypt_numbers(sequence, 0)

Decrypted result: QJDPDUGUIFOVNCFSTNBTPO with 0 shift
Decrypted result: PICOCTFTHENUMBERSMASON with 1 shift
Decrypted result: OHBNBSESGDMTLADQRLZRNM with 2 shift


### 13

Este ejercicio es igual a ROT13. Así que reutilizaremos la función ya creada.

> Decrypt `cvpbPGS{abg_gbb_onq_bs_n_ceboyrz}`

In [6]:
rot13("cvpbPGS{abg_gbb_onq_bs_n_ceboyrz}")

'picoCTF{not_too_bad_of_a_problem}'

### Easy Peasy

Este ejercicio hace referencia al cifrado OTP. El cifrado OTP (One-Time Pad) es un sistema de cifrado extremadamente seguro que utiliza una clave secreta aleatoria de números o letras que debe ser tan larga como el mensaje que se va a cifrar y que nunca debe reutilizarse. Para cifrar un mensaje, el emisor realiza una operación XOR (o exclusiva) entre cada letra (o carácter) del mensaje original y la correspondiente letra (o carácter) en la clave OTP. 

Ahora bien, existen algunas cosas en este ejercicio que nos pueden ayudar a romper el cifrado. La idea en el algoritmo de otp.py es que, cada vez que se envía un mensaje para encriptar, se actualiza la variable posición `key_position` para usar distintas porciones de la clave en la encriptación. En algún punto `key position` estará en una posición cerca de la longitud de la clave, que es 50 000. Si se envía un mensaje que sobrepasa o iguala la longitud, se realiza esta operación $stop = stop \% KEYLEN$. `stop` va junto a otra variable `start`, las cuales se encargan de indicar desde y hasta qué parte de la clave se va a usar para encriptar. `key_position` siempre se actualiza al valor de `stop` para continuar encriptando con la siguiente porción de clave. Por tanto, retomando la operación anterior, si `stop` sobrepasa o es igual al valor de 50 000 este se recalculará para que entre dentro de la longitud de la clave gracias al operador $(\%)$. 

Nuestro objetivo es usar la porción de clave con la que se encriptó el primer mensaje para reutilizar dicha porción y desencriptar el primer mensaje. Pero, al enviar el primer mensaje, `key_position` se actualiza a una posición $x$, que es determinada por la longitud del mensaje (lo que quiere decir que $x$ es la longitud del mensaje original). Así que, tenemos que hacer que `key_position` regrese a la posición 0; y esto lo podemos lograr si `key_position` = 50 000, ya que $50000 \% 50000 = 0$. Y para lograr que `key_position` sea igual a 50 000 debemos enviar un mensaje de longitud $50000-x$. 

> Cada letra del mensaje original se encripta en un valor hexadecimal de dos dígitos. Así que si obtenemos un FLAG encriptado de 64 caracteres, el mensaje original habrá sido de 32 caracteres ($x=32$)

In [114]:
from pwn import remote

conn = remote('mercury.picoctf.net', 36981)
print(conn.recvuntil('What data would you like to encrypt?'))

remaining_bytes = 50000 - 32    # flag is 32 bytes
while remaining_bytes >= 5000:
    print('[+] Sending 5000 bytes...')
    conn.send('a' * 5000 + '\r\n')
    remaining_bytes -= 5000
    conn.recvuntil('What data would you like to encrypt?')

print(f'[+] Sending {remaining_bytes} bytes...')
conn.send('a' * remaining_bytes + '\r\n')
print(conn.recvuntil('What data would you like to encrypt?'))

print('[+] Key offset is at 0. Sending aaaaaaaaaa...')
conn.send('a' * 32 + '\r\n')
print(conn.recvuntil('What data would you like to encrypt?'))

conn.close()
print("Done")

[x] Opening connection to mercury.picoctf.net on port 36981
[x] Opening connection to mercury.picoctf.net on port 36981: Trying 18.189.209.142
[+] Opening connection to mercury.picoctf.net on port 36981: Done


/var/folders/bf/74p6fnsx6v582xb04v67qg4r0000gn/T/ipykernel_2676/291742730.py:4: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  print(conn.recvuntil('What data would you like to encrypt?'))


b'******************Welcome to our OTP implementation!******************\nThis is the encrypted flag!\n5541103a246e415e036c4c5f0e3d415a513e4a560050644859536b4f57003d4c\n\nWhat data would you like to encrypt?'
[+] Sending 5000 bytes...


/var/folders/bf/74p6fnsx6v582xb04v67qg4r0000gn/T/ipykernel_2676/291742730.py:9: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  conn.send('a' * 5000 + '\r\n')
/var/folders/bf/74p6fnsx6v582xb04v67qg4r0000gn/T/ipykernel_2676/291742730.py:11: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  conn.recvuntil('What data would you like to encrypt?')


[+] Sending 5000 bytes...
[+] Sending 5000 bytes...
[+] Sending 5000 bytes...
[+] Sending 5000 bytes...
[+] Sending 5000 bytes...
[+] Sending 5000 bytes...
[+] Sending 5000 bytes...
[+] Sending 5000 bytes...
[+] Sending 4968 bytes...


/var/folders/bf/74p6fnsx6v582xb04v67qg4r0000gn/T/ipykernel_2676/291742730.py:14: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  conn.send('a' * remaining_bytes + '\r\n')
/var/folders/bf/74p6fnsx6v582xb04v67qg4r0000gn/T/ipykernel_2676/291742730.py:15: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  print(conn.recvuntil('What data would you like to encrypt?'))


b' Here ya go!\n595047303d1951034b3d19000437170c233d1902593d1951553d1903502c2e3d1904584f3d1900543d1904573d190253073d1900573d190505253d190259403d1959593d1904593d1950593d1903583d1951563d1900532e3d19070539123d1905044d3d19055335513d195056223d190758193d1904003d1950055b3d195005415d2b21263d1903593d190251093d1900053d1903023d3d3d190707332c3d1958523d190754303d1905023c493d1959025b3d190303333d195153033d1903563d1903023e3e08023d1905523d190056243d1900523d195956073d1900503d1907073d1903073d1904503d1950593d1904563d190302033d1903033d1951573d1950503d1950003d195903315c0e3d1907023d1902562d3d1904053d1907043d1903583d195859233c3d1902533d1958053d1904564c3d1903023d1904513d1950503d1950033d1956073d1950043d1905513d19505708553d1950073d1958523d1907553d1958003d1900503d19515735233d1950573d1958573d1907574d3d1903555c3d1902503d133f533d1951021a3d1900053d190758243d1950553d1905503d190454353d1950032e3d1951573d1900563d1950043d1951523d1905543d1902023d1959553d1905580d3d1958073d195953183d195957213d1950023d1900543d1905513d19595932

/var/folders/bf/74p6fnsx6v582xb04v67qg4r0000gn/T/ipykernel_2676/291742730.py:18: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  conn.send('a' * 32 + '\r\n')
/var/folders/bf/74p6fnsx6v582xb04v67qg4r0000gn/T/ipykernel_2676/291742730.py:19: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  print(conn.recvuntil('What data would you like to encrypt?'))


b' Here ya go!\n0346483f243d1959563d1907563d1903543d190551023d1959073d1902573d19\n\nWhat data would you like to encrypt?'
[*] Closed connection to mercury.picoctf.net port 36981
Done


Una vez regresamos a la posición 0, enviaremos un mensaje de igual longitud al primer mensaje para que se utilice exactamente la misma porción de clave para encriptar. El mensaje que enviaremos es arbitrario, para este caso el mensaje será muchas "a".

Entonces hemos encriptado dos mensajes `m1` **(FLAG)** y `m2` (aaa...), y los ciframos utilizando la misma clave `c`, obteniendo dos mensajes cifrados `r1` y `r2`:

- `R1: 5541103a246e415e036c4c5f0e3d415a513e4a560050644859536b4f57003d4c`
- `R2: 0346483f243d1959563d1907563d1903543d190551023d1959073d1902573d19`

Ahora, si deseamos recuperar `m1` a partir de `r1`, `r2`, y `m2`, podemos realizar lo siguiente:

1. Deshacemos la operación de cifrado para obtener `c`:
   - `c = r1 XOR m1` (deshaciendo la operación de cifrado en `r1`)
   - `c = r2 XOR m2` (deshaciendo la operación de cifrado en `r2`)

2. Igualamos las dos expresiones de `c` porque ambas representan la misma clave `c`:
   - `r1 XOR m1 = r2 XOR m2`

3. Ahora, si deseamos recuperar `m1`, podemos aislarlo en el lado izquierdo de la ecuación:
   - `m1 = r1 XOR r2 XOR m2`

En resumen, la ecuación `m1 = (r1 XOR r2) XOR m2` muestra cómo recuperar el mensaje original `m1` a partir de los mensajes cifrados `r1`, `r2`, y el mensaje conocido `m2` utilizando operaciones XOR y deshaciendo la operación de cifrado. Esto funciona debido a las propiedades del XOR y la relación entre los mensajes originales, los mensajes cifrados y la clave de cifrado.

In [9]:
r1 = 0x5541103a246e415e036c4c5f0e3d415a513e4a560050644859536b4f57003d4c
r2 = 0x0346483f243d1959563d1907563d1903543d190551023d1959073d1902573d19
m2 = 0x6161616161616161616161616161616161616161616161616161616161616161 # (aaaaaa ....)

m1_hex = (r1 ^ r2) ^ m2
print(hex(m1_hex))

0x3766396461323966343034393961393864623232303338306135373734366134


In [10]:
m1 = "3766396461323966343034393961393864623232303338306135373734366134"
m1_bytes = bytes.fromhex(m1)
m1_str = m1_bytes.decode("utf-8")
print("m1 como string:", m1_str)

m1 como string: 7f9da29f40499a98db220380a57746a4


### Spelling Quiz

El tipo de cifrado empleado en este ejercicio es un cifrado por sustitución monoalfabético, donde cada letra del alfabeto original se mapea a una única letra en el alfabeto cifrado. En este caso, el mapeo se realiza hacia un alfabeto de orden pseudoaleatorio. No hay manera de saber cuál es el orden de este alfabeto, pero si ejecutamos un análisis de frecuencia, podremos hallar algunas pistas.

Tomando en cuenta que en el inglés la letra que se repite más veces es la 'e', entonces con el análisis de frecuencia podremos saber cuál es la letra encriptada que más se repite y deducir que esa es la llave para la letra 'e'.

In [122]:

from collections import Counter

def get_most_freq_letters(file):
    with open(file, 'r') as f:
        c = Counter()
        for line in f:
            c += Counter(line[:-1])
    return c.most_common()

most_freq_letters = list(dict(get_most_freq_letters("./public/study-guide.txt")).keys())
print(most_freq_letters)

['r', 'w', 'x', 't', 'i', 'a', 'c', 'v', 'l', 'n', 'o', 'b', 'm', 's', 'f', 'd', 'q', 'u', 'y', 'p', 'g', 'e', 'k', 'z', 'j', 'h']


Cómo observamos, la letra que más se repite en el archivo __study_guide.txt__ es la letra `r`. Por lo que podemos deducir que la `r` corresponde a la letra `e`.

Además, sabemos que una de las palabras más frecuentes en inglés es `the`. En el flag hallamos algunas palabras encriptadas que podrían ser `the` como: `vfr`, `mid` y `exa`. Sin embargo, habíamos dicho que la letra `r` corresponde a la letra `e`. Por ello, `the` encriptado debería terminar con `r`. La única palabra que cumple esto es `vfr`; por lo cual también podemos deducir que: `v` = `t` y `f` = `h`. Así, con estas pistas podemos empezar un ataque para tratar de adivinar a qué palabra corresponde cada palabra encriptada del flag.

In [6]:
flag_file = open("./public/flag.txt", "r")
flag = flag_file.read()
flag_file.close()

common_words_file = open("./public/words.txt", "r")
common_words = common_words_file.read().split("\n")
common_words_file.close()

def get_all_repeated_letter_positions(word):
    letter_positions = {}
    for index, char in enumerate(word):
        if char in letter_positions:
            letter_positions[char].append(index)
        else:
            letter_positions[char] = [index]
    return letter_positions

def get_similar_words(ew, dictionary):
    similar_words = []
    for cw in common_words:
        if (len(cw) == len(ew)):
            count_cw = get_all_repeated_letter_positions(cw)
            count_ew = get_all_repeated_letter_positions(ew)
            if list(count_cw.values()) == list(count_ew.values()):
                keys_on_ew = [key for key in dictionary.values() if key in count_ew.keys()]
                if len(keys_on_ew)>0:
                    if all([key in count_cw.keys() for key in keys_on_ew]):
                        if all([count_cw[key] == count_ew[key] for key in keys_on_ew]):
                            similar_words.append(cw)
                else:
                    if all([key not in dictionary.values() for key in count_cw.keys()]):
                        similar_words.append(cw)
    return similar_words

def update_dictionary(dictionary, ew, w):
    for i, c in enumerate(ew):
        if (w[i] not in dictionary.values()):
            dictionary[c] = w[i]
    return dictionary

def decrypt_flag(flag, dictionary):
    result = ""
    for letter in flag:
        if letter in dictionary.keys():
            result += dictionary[letter]
        else:
            result += letter
    return result

dictionary = {'r': 'e', 'v': 't', 'f': 'h'} # inicializamos con los resultados del analisis de frecuencias
flag = decrypt_flag(flag, dictionary).split("_")

while str(input("Would you like to continue guessing?[y/n]: ")) == 'y':
    for i, ew in enumerate(flag):
        print(f"\nCurrent flag decrypted: {'_'.join(flag)}")
        similar_words = get_similar_words(ew, dictionary)
        if len(similar_words)>0:
            for sw in similar_words:
                print(f"{sw} -> {ew}?")
                x = str(input("Do you want to update dictionary with this word?"))
                if x == 'y':
                    dictionary = update_dictionary(dictionary, ew, sw)
                    flag[i] = decrypt_flag(ew, dictionary)
                    break    


Current flag decrypted: bechxba_the_mid_hosbem_ipec_exa_hoat_twcem
perhaps -> bechxba?

Current flag decrypted: perhaps_the_mid_hosbem_ipec_exa_hoat_twcem
the -> the?

Current flag decrypted: perhaps_the_mid_hosbem_ipec_exa_hoat_twcem
you -> mid?
now -> mid?
buy -> mid?
own -> mid?
big -> mid?
old -> mid?
low -> mid?
job -> mid?
box -> mid?
nov -> mid?
god -> mid?
log -> mid?
fun -> mid?
jul -> mid?
jun -> mid?
oil -> mid?
dog -> mid?

Current flag decrypted: perhaps_the_dog_hosbem_ipec_exa_hoat_twcem
hosted -> hosbem?

Current flag decrypted: perhaps_the_dog_hosped_ipec_exa_hoat_twcem
open -> ipec?
mpeg -> ipec?
spec -> ipec?
jpeg -> ipec?

Current flag decrypted: perhaps_the_dog_hosped_ipec_exa_hoat_twcem
era -> exa?
epa -> exa?
eva -> exa?

Current flag decrypted: perhaps_the_dog_hosped_ipec_exa_hoat_twcem

Current flag decrypted: perhaps_the_dog_hosped_ipec_exa_hoat_twcem
times -> twcem?
taken -> twcem?
types -> twcem?
takes -> twcem?
tried -> twcem?
taxes -> twcem?
tower -> twcem

### New Caesar

Ya hemos visto cómo funciona el cifrado César. Todo se basa en el desplazamiento acorde a una clave, que en este caso es una letra que puede ser entre la a - p. Para descifrar en cifrado César siempre recurrimos al proceso inverso. Por ejemplo, en el ejercicio de **Spelling quiz**, tuvimos que hacer el proceso inverso de $(x+k)\%26$, que es $(x-k)\%26$. Lo mismo sucede aquí, pero además de la operación modular, también existe una codificación en base 16 (b16). Así que tendremos que invertir la función de base 16 para decodificarla. Por último, para hallar el desplazamiento correcto, simplemente tendremos que iterar sobre todas las opciones de la 'a' a la 'p', hasta hallar un FLAG resultante que sea probablemente correcto.


In [157]:
ENCRYPTED_FLAG = "dcebcmebecamcmanaedbacdaanafagapdaaoabaaafdbapdpaaapadanandcafaadbdaapdpandcac"

ALPHABET = LOWER_ALPHABET[:16]

def b16_decode(b16):
    f = ""
    for i in range(0, len(b16), 2):
        x1 = ALPHABET.index(b16[i])
        x2 = ALPHABET.index(b16[i+1])
        b = "{0:04b}".format(x1) + "{0:04b}".format(x2)
        f += chr(int(b, 2))
    return f

def unshift(e, k):
    t1 = ord(e) + ord(ALPHABET[0])
    t2 = ord(k) + ord(ALPHABET[0])
    return ALPHABET[(t1 - t2) % len(ALPHABET)]

def decrypt_new_caesar(enc):
    print("Possible Flags:")
    for k in ALPHABET:
        dec = ""
        for e in enc:
            dec += unshift(e, k)
        flag = b16_decode(dec)
        if all(char in string.printable for char in flag): #chequeamos si hay chars no imprimibles
            print(f"---> Using key {k}: {flag}")
        
decrypt_new_caesar(ENCRYPTED_FLAG)

Possible Flags:
---> Using key n: et_tu?_07d5c0892c1438d2b32600e83dc2b0e5
---> Using key o: TcNcd.N/&S$R/'(!R #"'S!Q"!%//T'"SR!Q/T$
